# Random Search

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub

from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, reciprocal
from sklearn.metrics import f1_score
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Factored project/Models/newdata.csv')
data = data.drop(columns=['Unnamed: 0'], axis=1)
Y = data['target']
X = data.drop(columns=['target'], axis=1)

In [ ]:
def Embedding_function(X):  
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  X = embed(X)
  X = X.numpy()
  return X

In [ ]:
model = SVC(random_state=40)
# parameters = {'l1_ratio': uniform(0, 1)}
# # parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# # distributions = dict(gamma = reciprocal(0.001, 0.1))
np.random.seed(123)
g_range = np.random.uniform(0.0, 0.3, 5).astype(float)
C_range = np.random.normal(1, 0.1, 5).astype(float)
hyperparameters = {'gamma': list(g_range), 
                    'C': list(C_range)}

svm_clf = RandomizedSearchCV(model, param_distributions=hyperparameters, random_state=0, scoring='f1', n_iter=20)
Resultados = {'Set':['Train', 'Test']}
for i in X.columns:
  x_em = Embedding_function(X.loc[:,i])
  # Partición de los datos
  x_train, x_test, y_train, y_test = train_test_split(x_em, Y, test_size=0.2, random_state=40)
  # Buscar hiperparámetros que maximizan la metrica f1
  search = svm_clf.fit(x_train, y_train)
  var = 'model_SVM_{}'.format(i)
  joblib.dump(search, '/content/drive/My Drive/Factored project/Hyperparameter/{}.pkl'.format(var))
  y_pre = search.predict(x_test)
  Resultados[i] = [search.best_score_, f1_score(y_test, y_pre)]

print(Resultados)

NameError: ignored

In [ ]:
# Identify optimal hyperparameter values
best_gamma  = svm_clf.best_params_['gamma']
best_C      = svm_clf.best_params_['C']
 
print("The best performing gamma value is: {:5.2f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))

The best performing gamma value is:  0.21
The best performing C value is:  0.84


In [ ]:
R_hyp = pd.DataFrame(data=Resultados)
R_hyp.to_csv('/content/drive/My Drive/Factored project/Hyperparameter/R_SVM_whyp.csv')
R_hyp.head()

,Set,text,NoRuido,PRE,URLS,HTML,EMOJIS,TODO
0,Train,0.773862,0.771124,0.769331,0.770871,0.773862,0.773862,0.769331
1,Test,0.763607,0.772727,0.771679,0.772727,0.763607,0.763607,0.771679


In [ ]:
data.head()

,text,target,NoRuido,PRE,URLS,HTML,EMOJIS,TODO
0,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,deeds reason earthquake may allah forgive us,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this #earthquake M...,deeds reason earthquake may allah forgive us
1,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,residents asked shelter place notified officer...,All residents asked to 'shelter in place' are ...,All residents asked to 'shelter in place' are ...,All residents asked to 'shelter in place' are ...,residents asked shelter place notified officer...
3,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",13000 people receive wildfires evacuation orde...,"13,000 people receive #wildfires evacuation or...","13,000 people receive #wildfires evacuation or...","13,000 people receive #wildfires evacuation or...",13000 people receive wildfires evacuation orde...
4,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,got sent photo ruby alaska smoke wildfires pou...,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby #Alaska as ...,got sent photo ruby alaska smoke wildfires pou...
